# 生成式網絡

循環神經網絡（Recurrent Neural Networks, RNNs）及其門控單元變體，例如長短期記憶單元（Long Short Term Memory Cells, LSTMs）和門控循環單元（Gated Recurrent Units, GRUs），提供了一種語言建模的機制，也就是說，它們可以學習單詞的排列順序，並對序列中的下一個單詞進行預測。這使得我們可以使用 RNNs 來完成**生成任務**，例如普通文本生成、機器翻譯，甚至是圖像描述生成。

在上一單元中討論的 RNN 結構中，每個 RNN 單元會輸出下一個隱藏狀態。然而，我們也可以為每個循環單元添加另一個輸出，這樣就可以輸出一個**序列**（其長度與原始序列相等）。此外，我們還可以使用不在每一步接受輸入的 RNN 單元，而僅僅接受一個初始狀態向量，然後生成一個輸出序列。

在這個筆記本中，我們將專注於幫助我們生成文本的簡單生成模型。為了簡化，我們將構建一個**字符級網絡**，逐字母生成文本。在訓練過程中，我們需要使用一些文本語料庫，並將其拆分為字母序列。


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## 建立字符詞彙表

要建立字符級生成網絡，我們需要將文本拆分為單個字符，而不是單詞。我們之前使用的 `TextVectorization` 層無法做到這一點，因此我們有以下兩個選擇：

* 手動載入文本並自行進行分詞，如 [這個官方 Keras 範例](https://keras.io/examples/generative/lstm_character_level_text_generation/) 中所示
* 使用 `Tokenizer` 類進行字符級分詞。

我們會選擇第二個選項。`Tokenizer` 也可以用於分詞成單詞，因此可以輕鬆地從字符級分詞切換到單詞級分詞。

要進行字符級分詞，我們需要傳入 `char_level=True` 參數：


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

我們還希望使用一個特殊的標記來表示**序列結束**，我們將其稱為 `<eos>`。讓我們手動將其添加到詞彙表中：


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

現在，要將文本編碼為數字序列，我們可以使用：


In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## 訓練生成式 RNN 來生成標題

我們將以以下方式訓練 RNN 來生成新聞標題。在每一步中，我們會取一個標題，將其輸入到 RNN 中，並對於每個輸入的字符，要求網絡生成下一個輸出的字符：

![顯示 RNN 生成單詞 'HELLO' 的示例圖片。](../../../../../lessons/5-NLP/17-GenerativeNetworks/images/rnn-generate.png)

對於序列中的最後一個字符，我們會要求網絡生成 `<eos>` 標記。

我們在這裡使用的生成式 RNN 的主要不同之處在於，我們會從 RNN 的每一步輸出中取結果，而不僅僅是從最後一個單元格中取結果。這可以通過為 RNN 單元格指定 `return_sequences` 參數來實現。

因此，在訓練過程中，網絡的輸入將是一個特定長度的編碼字符序列，而輸出將是一個相同長度的序列，但向後偏移一個元素並以 `<eos>` 結尾。小批量數據將由若干這樣的序列組成，我們需要使用**填充**來對齊所有序列。

現在讓我們創建一些函數來幫助我們轉換數據集。由於我們希望在小批量層級上填充序列，我們會先通過調用 `.batch()` 將數據集分批，然後使用 `map` 來進行轉換。因此，轉換函數將以整個小批量作為參數：


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

一些重要的事情我們在這裡做：
* 我們首先從字串張量中提取實際文本
* `text_to_sequences` 將字串列表轉換為整數張量列表
* `pad_sequences` 將這些張量填充到它們的最大長度
* 最後，我們對所有字符進行 one-hot 編碼，並進行移位和附加 `<eos>`。我們很快就會看到為什麼需要 one-hot 編碼的字符

然而，這個函數是 **Pythonic** 的，也就是說它無法自動轉換為 Tensorflow 的計算圖。如果我們直接在 `Dataset.map` 函數中使用這個函數，會出現錯誤。我們需要使用 `py_function` 包裝器來封裝這個 Pythonic 調用：


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **注意**：區分 Pythonic 和 Tensorflow 的轉換函數可能看起來有點複雜，你可能會疑惑為什麼我們不在將數據集傳遞給 `fit` 之前，使用標準的 Python 函數進行轉換。雖然這確實是可行的，但使用 `Dataset.map` 有一個巨大的優勢，因為數據轉換管道是通過 Tensorflow 的計算圖執行的，這可以利用 GPU 的計算能力，並且減少了在 CPU 和 GPU 之間傳遞數據的需求。

現在我們可以建立生成器網絡並開始訓練。它可以基於我們在上一單元討論過的任何循環單元（簡單的、LSTM 或 GRU）。在我們的例子中，我們將使用 LSTM。

由於網絡以字符作為輸入，且詞彙表的大小相對較小，我們不需要嵌入層，直接使用 one-hot 編碼的輸入進入 LSTM 單元即可。輸出層將是一個 `Dense` 分類器，用於將 LSTM 的輸出轉換為 one-hot 編碼的標籤數字。

此外，因為我們處理的是可變長度的序列，我們可以使用 `Masking` 層來創建一個掩碼，忽略字符串中填充的部分。這並不是絕對必要的，因為我們對 `<eos>` 標籤之後的內容並不特別感興趣，但我們會使用它來獲得一些使用這類型層的經驗。`input_shape` 將是 `(None, vocab_size)`，其中 `None` 表示可變長度的序列，而輸出形狀也是 `(None, vocab_size)`，正如你可以從 `summary` 中看到的那樣。


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## 生成輸出

現在我們已經訓練了模型，接下來我們希望使用它來生成一些輸出。首先，我們需要一種方法來解碼由一系列標記數字表示的文本。為此，我們可以使用 `tokenizer.sequences_to_texts` 函數；然而，這個方法在字符級標記化時效果不佳。因此，我們將從 tokenizer 中提取標記字典（稱為 `word_index`），構建一個反向映射，並編寫我們自己的解碼函數：


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

現在，我們開始進行生成。我們會從某個字串 `start` 開始，將其編碼成一個序列 `inp`，然後在每一步調用我們的網絡來推斷下一個字符。

網絡的輸出 `out` 是一個包含 `vocab_size` 個元素的向量，代表每個標記的概率。我們可以通過使用 `argmax` 找到最有可能的標記編號。接著，我們將這個字符附加到已生成的標記列表中，並繼續生成過程。這個生成一個字符的過程會重複執行 `size` 次，以生成所需數量的字符。如果在過程中遇到 `eos_token`，我們會提前終止生成。


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## 在訓練期間抽樣輸出

由於我們沒有任何有用的指標，例如 *準確率*，唯一能判斷模型是否有所改善的方法就是在訓練期間**抽樣**生成的字串。為了實現這一點，我們會使用**回調函數**，即可以傳遞給 `fit` 函數的函數，並在訓練過程中定期被調用。


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


這個例子已經生成了一些相當不錯的文本，但仍有幾個方面可以進一步改進：

* **更多文本**。我們只使用了標題來進行任務，但你可能想嘗試使用完整的文本。請記住，RNN在處理長序列時表現並不太好，因此可以考慮將它們拆分成較短的句子，或者始終在固定的序列長度（例如，`num_chars`，假設為256）上進行訓練。你可以嘗試將上面的例子改造成這樣的架構，並參考[官方 Keras 教程](https://keras.io/examples/generative/lstm_character_level_text_generation/)作為靈感。

* **多層 LSTM**。可以嘗試使用2層或3層的LSTM單元。如我們在前一單元提到的，每一層LSTM會從文本中提取特定的模式，而在字符級生成器的情況下，我們可以預期較低層的LSTM負責提取音節，而較高層則負責提取單詞和單詞組合。這可以通過向LSTM構造函數傳遞層數參數來簡單實現。

* 你可能還想嘗試使用**GRU單元**，看看哪種表現更好，或者嘗試**不同的隱藏層大小**。隱藏層過大可能導致過擬合（例如，網絡會學習到精確的文本），而過小的大小可能無法產生良好的結果。


## 軟性文本生成與溫度

在之前 `generate` 的定義中，我們總是選擇機率最高的字符作為生成文本中的下一個字符。這導致文本經常在相同的字符序列之間「循環」出現，如以下例子所示：
```
today of the second the company and a second the company ...
```

然而，如果我們查看下一個字符的機率分佈，可能會發現幾個最高機率之間的差距並不大，例如一個字符的機率是 0.2，另一個是 0.19，等等。例如，在尋找序列 *play* 的下一個字符時，下一個字符可能同樣有可能是空格，或者是 **e**（如單詞 *player* 中的情況）。

這讓我們得出一個結論：選擇機率最高的字符並不總是「公平」的，因為選擇第二高的字符仍然可能生成有意義的文本。更明智的做法是從網絡輸出的機率分佈中**抽樣**字符。

這種抽樣可以使用 `np.multinomial` 函數來完成，該函數實現了所謂的**多項分佈**。下面定義了一個實現這種**軟性**文本生成的函數：


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

我們引入了一個名為 **temperature** 的參數，用於指示我們應該多大程度地堅持最高概率。如果 temperature 是 1.0，我們進行公平的多項式抽樣，而當 temperature 趨於無窮大時，所有概率變得相等，我們隨機選擇下一個字符。在下面的例子中，我們可以觀察到當我們將 temperature 增加得太多時，文本變得毫無意義，而當它接近 0 時，則類似於「循環」的硬生成文本。



---

**免責聲明**：  
本文件已使用人工智能翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。儘管我們致力於提供準確的翻譯，但請注意，自動翻譯可能包含錯誤或不準確之處。原始語言的文件應被視為權威來源。對於重要資訊，建議使用專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或錯誤解釋概不負責。
